In [1]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [26]:
check = client.query(
"""
    SELECT 
        COUNT(DISTINCT exposure_crn) AS exposed_crns, 
        COUNT(DISTINCT shopper_identification_number) AS converted_crns
    FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_plus_history` 
    WHERE booking_id = "WOW20004911" 
""").result()
check_df = check.to_dataframe()
chime.success()
check_df

,exposed_crns,converted_crns
0,1005039,684124


In [27]:
check = client.query(
    """
    SELECT 
        perf.*, 
        
    FROM `gcp-wow-cart-data-dev-d4d7.davide.digital_exposure_based_campaign_performance` perf, 
    UNNEST(skus) AS sku
    WHERE exposed_conversion_rate >= 0.85
    AND booking_id = "WOW20004911"
    """).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,campaign_start_date,campaign_end_date,n_days,media_types,skus,n_skus,exposed_conversion_rate,not_exposed_conversion_rate,conversion_rate_uplift_perc,conversion_rate_uplift_pp,spend_per_exposed_converted_shopper,spend_per_non_exposed_converted_shopper,spend_per_converted_shopper_uplift_perc,spend_per_converted_shopper_uplift_pp
0,WOW20004911,2023-11-01,2023-11-28,27,"[Display Customer Segment, Display Search]","[387350, 387435, 388036]",3,0.878977,0.674014,0.304092,0.204962,3.119713442,1.943358181,0.605320868,1.176355261


In [51]:
check = client.query(
    """
    SELECT *
    FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_sku_level_v2` perf
    WHERE exposure_context = "Not Exposed"
    AND basket_key IS NULL
    LIMIT 10
    """).result()
check_df = check.to_dataframe()
chime.success()
check_df

,impression_date,booking_id,exposure_context,exposure_crn,order_context,converted_shopper_identification_number,start_txn_time,basket_key,article,tot_net_incld_gst,exposure_to_purchase_window
0,NaT,WOW20004728,Not Exposed,3300000000008604409,None,None,NaT,None,None,None,7 Days
1,NaT,WOW20004728,Not Exposed,3300000000007929445,None,None,NaT,None,None,None,7 Days
2,NaT,WOW20004728,Not Exposed,1100000000135383872,None,None,NaT,None,None,None,7 Days
3,NaT,WOW20004728,Not Exposed,1100000000002104560,None,None,NaT,None,None,None,7 Days
4,NaT,WOW20004728,Not Exposed,1000000000002777167,None,None,NaT,None,None,None,7 Days
5,NaT,WOW20004728,Not Exposed,3300000000002095003,None,None,NaT,None,None,None,7 Days
6,NaT,WOW20004728,Not Exposed,3300000000008547758,None,None,NaT,None,None,None,7 Days
7,NaT,WOW20004728,Not Exposed,3300000000001864073,None,None,NaT,None,None,None,7 Days
8,NaT,WOW20004728,Not Exposed,1100000000162777290,None,None,NaT,None,None,None,7 Days
9,NaT,WOW20004728,Not Exposed,1100000000005214911,None,None,NaT,None,None,None,7 Days


In [65]:
check = client.query(
    """
CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.digital_exposure_based_conversion_and_sales` AS (
    with exposed_sales AS (
        SELECT DISTINCT 
            booking_id, 
            exposure_crn, 
            basket_key, 
            article,
            tot_net_incld_gst 
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_sku_level_v2`  
        WHERE exposure_context = "Exposed"
    ),
    not_exposed_sales AS (
        SELECT DISTINCT 
            not_e.booking_id,
            not_e.exposure_crn, 
            not_e.basket_key, 
            not_e.article,
            not_e.tot_net_incld_gst 
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_sku_level_v2` not_e
        LEFT JOIN exposed_sales 
            ON 
                not_e.booking_id = exposed_sales.booking_id 
                AND not_e.exposure_crn = exposed_sales.exposure_crn 
                AND not_e.basket_key = exposed_sales.basket_key
                AND not_e.article = exposed_sales.article
                AND not_e.tot_net_incld_gst = exposed_sales.tot_net_incld_gst
        WHERE exposure_context = "Not Exposed"  
        AND exposed_sales.basket_key IS NULL
    ),
    sales_unioned AS (
        SELECT 
            exposed_sales.*, 
            "Exposed" AS exposure_context
        FROM exposed_sales 
        UNION ALL 
        SELECT 
            not_exposed_sales.*, 
            "Not Exposed" AS exposure_context 
        FROM not_exposed_sales
    ),
    sales_aggregated AS (
        SELECT 
            booking_id, 
            COUNT(DISTINCT CASE WHEN exposure_context = "Not Exposed" THEN basket_key ELSE NULL END) AS n_not_exposed_baskets_with_promo,
            COUNT(DISTINCT CASE WHEN exposure_context = "Exposed" THEN basket_key ELSE NULL END) AS n_exposed_baskets_with_promo,
            SUM(CASE WHEN exposure_context = "Not Exposed" THEN tot_net_incld_gst ELSE 0 END) AS sum_of_not_exposed_promo_sales,
            SUM(CASE WHEN exposure_context = "Exposed" THEN tot_net_incld_gst ELSE 0 END) AS sum_of_exposed_promo_sales,
        FROM sales_unioned 
        GROUP BY 1
    ), 
    conversion_rates AS (
        SELECT
            booking_id, 
            COUNT(DISTINCT CASE WHEN exposure_context = "Not Exposed" THEN exposure_crn ELSE NULL END) AS n_not_exposed, 
            COUNT(DISTINCT CASE WHEN exposure_context = "Exposed" THEN exposure_crn ELSE NULL END) AS n_exposed,
            COUNT(DISTINCT CASE WHEN converted_shopper_identification_number IS NOT NULL AND exposure_context = "Not Exposed" THEN exposure_crn ELSE NULL END) / COUNT(DISTINCT CASE WHEN exposure_context = "Not Exposed" THEN exposure_crn ELSE NULL END) AS not_exposed_conversion_rate,
            COUNT(DISTINCT CASE WHEN converted_shopper_identification_number IS NOT NULL AND exposure_context = "Exposed" THEN exposure_crn ELSE NULL END) / COUNT(DISTINCT CASE WHEN exposure_context = "Exposed" THEN exposure_crn ELSE NULL END) AS exposed_conversion_rate, 
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_sku_level_v2` 
        GROUP BY 1
        ORDER BY 3 DESC
    )
    SELECT 
        conversion_rates.booking_id, 
        campaign_start_date,
        campaign_end_date,
        n_days,
        media_types,
        subcats, 
        skus,
        n_skus,
        n_exposed, 
        n_not_exposed, 
        exposed_conversion_rate,
        not_exposed_conversion_rate,
        n_exposed_baskets_with_promo / n_exposed AS n_exposed_baskets_per_exposed_shopper,
        n_not_exposed_baskets_with_promo / n_not_exposed AS n_not_exposed_baskets_per_not_exposed_shopper,
        sum_of_exposed_promo_sales / n_exposed AS exposed_promo_sales_per_exposed_shopper, 
        sum_of_not_exposed_promo_sales / n_not_exposed AS not_exposed_promo_sales_per_not_exposed_shopper
    FROM conversion_rates 
    LEFT JOIN sales_aggregated
        ON conversion_rates.booking_id = sales_aggregated.booking_id 
    LEFT JOIN (
            SELECT 
                booking_number, 
                campaign_start_date, 
                campaign_end_date,
                DATE_DIFF(campaign_end_date, campaign_start_date, DAY) AS n_days,  
                ARRAY_AGG(DISTINCT media_type ORDER BY media_type) AS media_types  
            FROM gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns 
            GROUP BY 1,2,3
    ) media ON media.booking_number = conversion_rates.booking_id 
    LEFT JOIN (
            SELECT 
                booking_number, 
                ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS skus,
                ARRAY_LENGTH(ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string)) AS n_skus
            FROM gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns 
            GROUP BY 1
    ) articles ON articles.booking_number = conversion_rates.booking_id 
    LEFT JOIN (
            SELECT 
                booking_number, 
                ARRAY_AGG(DISTINCT Sub_CategoryDescription ORDER BY Sub_CategoryDescription) AS subcats
            FROM gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns campaigns
            LEFT JOIN `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_masterdata_view.dim_article_master_v` am 
                ON TRIM(campaigns.individual_product_string) = am.Article 
            WHERE am.SalesOrg = 1005 
            GROUP BY 1
    ) subcats_t ON subcats_t.booking_number = conversion_rates.booking_id 
    ORDER BY 11 DESC
);
    """).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [66]:
check = client.query(
"""SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.digital_exposure_based_conversion_and_sales` ORDER BY exposed_conversion_rate DESC"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,campaign_start_date,campaign_end_date,n_days,media_types,subcats,skus,n_skus,n_exposed,n_not_exposed,exposed_conversion_rate,not_exposed_conversion_rate,n_exposed_baskets_per_exposed_shopper,n_not_exposed_baskets_per_not_exposed_shopper,exposed_promo_sales_per_exposed_shopper,not_exposed_promo_sales_per_not_exposed_shopper
0,WOW20004382,2023-11-01,2023-12-26,55,"[Catalogue, Display Search]","[GM OTHER, SOFT DRINKS - BOTTLES & CANS, SOFT DRINKS - COLD DRINK, SOFT DRINKS - MINERAL WATER, SOFT DRINKS - SPORTS, SOFT DRINKS - WATER, UK AND IRISH FOODS]","[136874, 137848, 140090, 140092, 140640, 15917, 159796, 159797, 159798, 159799, 159801, 162961, 162962, 176210, 178132, 178416, 178822, 180910, 180941, 215509, 220986, 221011, 221494, 221678, 221713, 221774, 221822, 237909, 238400, 238548, 238684, 242089, 242317, 242611, 243692, 243760, 243795, 244282, 244370, 244826, 244886, 244993, 245007, 245069, 245582, 246323, 260366, 260368, 262843, 279908, 285384, 286997, 287016, 290133, 290344, 313019, 315221, 317387, 317388, 318562, 322924, 32731, 32812, 332730, 332732, 333023, 34691, 360462, 362262, 365186, 365189, 365190, 365548, 369725, 369795, 370757, 38121, 42605, 45533, 51557, 51569, 51570, 52547, 52558, 62170, 623033, 623034, 623069, 669379, 669683, 672966, 676160, 679121, 679122, 723452, 723453, 725866, 750733, 760280, 77938, ...]",122,517884,995499,0.544466,0.418644,1.926568,1.307389,25.439613215,17.751529394
1,WOW20004380,2023-10-04,2023-10-24,20,[Display Search],[BANANA],[133211],1,21781,967869,0.503099,0.346558,1.182820,0.704642,5.259982094,2.869911868
2,WOW20004498,2024-01-03,2024-02-20,48,"[Display Search, Results Page Display]","[CORDIAL, LUNCHBOX DRINKS, SOFT DRINKS - BOTTLES & CANS, SOFT DRINKS - COLD DRINK, SOFT DRINKS - ICED TEA, SOFT DRINKS - MINERAL WATER, SOFT DRINKS - MIXERS, SOFT DRINKS - SPORTS, SOFT DRINKS - WATER]","[12011, 121164, 130504, 140657, 141053, 152235, 152303, 153052, 155291, 159878, 159946, 160144, 160239, 173487, 175763, 189114, 190931, 201688, 201689, 219582, 221029, 221228, 221517, 221667, 223815, 225840, 227739, 227782, 229460, 229607, 229664, 229753, 230839, 230840, 231413, 231452, 231553, 231593, 245411, 253760, 253762, 253766, 253770, 253771, 253773, 253808, 253811, 253828, 262841, 275313, 275744, 277314, 280288, 280394, 280988, 282425, 282467, 283456, 286195, 286208, 286345, 288082, 290319, 290348, 290705, 291865, 292947, 296511, 296717, 30441, 30446, 30450, 30486, 305223, 305224, 317793, 32673, 335129, 335559, 341058, 35766, 35808, 36171, 362445, 362446, 362459, 362461, 362464, 362472, 38433, 38435, 40895, 430139, 430255, 442169, 442182, 442185, 442187, 442190, 442191, ...]",144,82119,1304640,0.469246,0.358541,1.328341,0.921921,19.271105347,10.608091757
3,WOW20004608,2023-10-04,2023-10-31,27,"[Catalogue, Display Aisle and Category, Display Reach, Display Search, Home Page]","[CONFEC - GIFTING, CONFEC ON THE GO - CHOCOLATE & SUGAR, CONFEC PORTION PACKS - CHOCOLATE & SUGAR, CONFEC SHARING - CHOCOLATE, CONFEC SHARING - CHOCOLATE BLOCKS, CONFEC SHARING - SUGAR]","[103110, 109586, 136390, 136391, 173152, 180111, 194882, 195000, 210601, 210630, 211963, 211973, 211982, 213155, 229648, 230698, 231257, 232517, 232519, 233004, 233525, 233587, 233802, 233803, 233892, 234585, 234699, 250746, 253430, 254447, 254451, 254672, 263728, 274979, 275279, 275826, 277647, 278242, 280854, 289426, 298383, 304748, 306776, 309495, 313239, 314114, 317394, 318604, 320107, 320110, 324126, 326628, 360300, 46053, 697802, 715904, 77424, 779407, 81287, 813881, 814479, 814558, 814894, 815197, 825105, 829138, 829144, 829163, 829166, 829175, 840556, 87143, 87144, 87145, 91151, 953973, 98361, NPD]",78,655346,446386,0.352611,0.297346,0.674798,0.541245,4.056724158,2.925979623
4,WOW20004121,2024-04-03,2024-04-30,27,"[Display Reach, Results Page Display]",[MEXICAN FOODS],"[106518, 108174, 14040, 15105, 169527, 169871, 187333, 19977, 2037, 210693, 228154, 229703, 230731, 230870, 231416, 231887, 233280, 248677, 248678, 248

In [33]:
check = client.query(
    """
    SELECT 
        booking_id,
        COUNT(DISTINCT CASE WHEN converted_shopper_identification_number IS NULL THEN exposed_shopper_identification_number) AS exposed_shoppers
        COUNT(DISTINCT converted_shopper_identification_number) AS converts
    FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_sku_level_v2` perf
    WHERE booking_id = "WOW20004911"
    GROUP BY 1,2
    ORDER BY 3 DESC 
    LIMIT 10
    """).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,exposed_shopper_identification_number,converts
0,WOW20004911,1100000000002388548,1
1,WOW20004911,1100000000033768151,1
2,WOW20004911,3300000000003866854,1
3,WOW20004911,1100000000027043702,1
4,WOW20004911,1000000000001336355,1
5,WOW20004911,1100000000108423852,1
6,WOW20004911,3300000000007876363,1
7,WOW20004911,3300000000001211609,1
8,WOW20004911,3300000000000809754,1
9,WOW20004911,1100000000090233132,1


In [71]:
check = client.query(
    """
SELECT 
    order_context,
    COUNT(DISTINCT shopper_identification_number) AS n_crns 
FROM `gcp-wow-cart-data-dev-d4d7.davide.test_transactions`  
WHERE article = "133211" 
AND start_txn_date >= DATE("2023-10-04")
AND start_txn_date <= DATE("2023-10-24")
GROUP BY 1
""").result()
check_df = check.to_dataframe()
chime.success()
check_df

,order_context,n_crns
0,Instore Order,2271317
1,Online Order,388890


In [ ]:
check = client.query(
    """
--CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.digital_exposure_based_campaign_performance` AS 
with step_one AS (
    SELECT 
        booking_id, 
        SAFE_DIVIDE(COUNT(DISTINCT CASE WHEN  THEN shopper_identification_number ELSE NULL END)  , COUNT(DISTINCT CASE WHEN cohort = "Exposed" THEN exposure_crn ELSE NULL END)) AS exposed_conversion_rate,
        SAFE_DIVIDE(COUNT(DISTINCT CASE WHEN cohort <> "Exposed" THEN shopper_identification_number ELSE NULL END) , COUNT(DISTINCT CASE WHEN cohort <> "Exposed" THEN exposure_crn ELSE NULL END)) AS not_exposed_conversion_rate, 
        SAFE_DIVIDE(SUM(CASE WHEN cohort = "Exposed" THEN total_converted_spend ELSE 0 END) , COUNT(DISTINCT CASE WHEN cohort = "Exposed" THEN shopper_identification_number ELSE NULL END)) AS spend_per_exposed_converted_shopper,
        SAFE_DIVIDE(SUM(CASE WHEN cohort <> "Exposed" THEN total_converted_spend ELSE 0 END) , COUNT(DISTINCT CASE WHEN cohort <> "Exposed" THEN shopper_identification_number ELSE NULL END)) AS spend_per_non_exposed_converted_shopper
    FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_sku_level_v2`
    GROUP BY 1
    ORDER BY 1
), step_two AS (
    SELECT 
        booking_id, 
        exposed_conversion_rate, 
        not_exposed_conversion_rate, 
        SAFE_DIVIDE(exposed_conversion_rate , not_exposed_conversion_rate) - 1 AS conversion_rate_uplift_perc,
        exposed_conversion_rate - not_exposed_conversion_rate AS conversion_rate_uplift_pp,
        spend_per_exposed_converted_shopper,
        spend_per_non_exposed_converted_shopper,
        SAFE_DIVIDE(spend_per_exposed_converted_shopper , spend_per_non_exposed_converted_shopper) - 1 AS spend_per_converted_shopper_uplift_perc,
        spend_per_exposed_converted_shopper - spend_per_non_exposed_converted_shopper AS spend_per_converted_shopper_uplift_pp
FROM step_one
)
SELECT 
    booking_id, 
    campaign_start_date,
    campaign_end_date,
    n_days,
    media_types,
    skus,
    n_skus,
    exposed_conversion_rate, 
    not_exposed_conversion_rate, 
    conversion_rate_uplift_perc,
    conversion_rate_uplift_pp,
    spend_per_exposed_converted_shopper,
    spend_per_non_exposed_converted_shopper,
    spend_per_converted_shopper_uplift_perc,
    spend_per_converted_shopper_uplift_pp
FROM step_two
LEFT JOIN (
        SELECT 
            booking_number, 
            campaign_start_date, 
            campaign_end_date,
            DATE_DIFF(campaign_end_date, campaign_start_date, DAY) AS n_days, 
            
            ARRAY_AGG(DISTINCT media_type ORDER BY media_type) AS media_types  
        FROM gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns 
        GROUP BY 1,2,3
) media ON media.booking_number = step_two.booking_id 
LEFT JOIN (
        SELECT 
            booking_number, 
            ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS skus,
            ARRAY_LENGTH(ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string)) AS n_skus
        FROM gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns 
        GROUP BY 1
) articles ON articles.booking_number = step_two.booking_id 
ORDER BY booking_id

""").result()
check_df = check.to_dataframe()
chime.success()
check_df

""
